In [1]:
import pandas as pd
from astroquery.mast import Observations
from astroquery.exceptions import RemoteServiceError
import os
from astropy.io import fits
import shutil
import logging 
import time 

logging.getLogger('astroquery').setLevel(logging.WARNING)
confirmed_planets = pd.read_csv("confirmed_planets.csv")
false_positives = pd.read_csv("false_positives.csv")
confirmed_log_file = './confirmed_index_log.txt'
false_log_file = './false_index_log.txt'

In [2]:
def process_and_save(tic_id_list, save_dir):
    retry = 3
    for tries in range(retry):
        try:
            print("Processing and Saving Data Start...")
            for _ in range(3):
                try:
                    print("Creating Observation Table")
                    obs_table = Observations.query_criteria(provenance_name="QLP", target_name=tic_id_list)
                    data_products = Observations.get_product_list(obs_table)
                    print("Attempting to download...")
                    download_lc = Observations.download_products(data_products)
                    break
                except RemoteServiceError as e:
                    print(f"Error: {e}. Retrying in 3 seconds...")
                    time.sleep(3)
                except Error as e:
                    print(f"An unexpected Error: {e}.")

            for product in download_lc["Local Path"]:
                if product is not None:
                    try:
                        with fits.open(product) as hdulist:
                            lc_data = hdulist[1].data
                            lc_df = pd.DataFrame(lc_data)
                            tic_id = hdulist[0].header['TICID']
                            lc_df.to_csv(os.path.join(save_dir, f"{tic_id}_lightcurve.csv"), index=False)
                    except (OSError, TypeError) as e:
                        print(f"Error processing file {product}: {e}")
            break  # break if successful
        except Exception as e:
            print(f"Error fetching data on attempt {tries + 1}/{retry}: {e}")
            sleep(5)  # Wait before retrying
            continue
            
def get_last_index(log_file):
    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            index = int(f.read().strip())
    else:
        index = 0
    return index
def update_log(log_file, index):
    with open(log_file, 'w') as f:
        f.write(str(index))

In [3]:
def download_and_zip_lc_batches(batch_size, batch_number):
    confirmed_last_index = get_last_index(confirmed_log_file)
    false_last_index = get_last_index(false_log_file)

    confirmed_tic_ids = confirmed_planets["TIC ID"].tolist()[confirmed_last_index:confirmed_last_index + batch_size]
    false_tic_ids = false_positives["TIC ID"].tolist()[false_last_index:false_last_index + batch_size]

    if len(false_tic_ids) < batch_size:
        false_tic_ids = false_positives["TIC ID"].tolist()[false_last_index:]

    confirmed_dir = "confirmed"
    false_dir = "false"

    os.makedirs(confirmed_dir, exist_ok=True)
    os.makedirs(false_dir, exist_ok=True)

    process_and_save(confirmed_tic_ids, confirmed_dir)
    print(f"Done confirmed batch {batch_number} sleeping for 10 seconds")
    time.sleep(10)

    if len(false_tic_ids) > 0:
        process_and_save(false_tic_ids, false_dir)
        print(f"Done false batch {batch_number} sleeping for 10 seconds")
        time.sleep(10)

    confirmed_zip = f"confirmed_lightcurves_{batch_number}.zip"
    false_zip = f"false_lightcurves_{batch_number}.zip"

    shutil.make_archive(confirmed_zip.replace(".zip", ""), "zip", confirmed_dir)
    if len(false_tic_ids) > 0:
        shutil.make_archive(false_zip.replace(".zip", ""), "zip", false_dir)

    print(f"Zipped confirmed lightcurves saved to: {confirmed_zip}")
    if len(false_tic_ids) > 0:
        print(f"Zipped false lightcurves saved to: {false_zip}")

    # Remove files after zipping
    for file in os.listdir(confirmed_dir):
        file_path = os.path.join(confirmed_dir, file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(f"Error: {e}")

    for file in os.listdir(false_dir):
        file_path = os.path.join(false_dir, file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(f"Error: {e}")

    update_log(confirmed_log_file, confirmed_last_index + batch_size)
    if len(false_tic_ids) > 0:
        update_log(false_log_file, false_last_index + len(false_tic_ids))
    else:
        update_log(false_log_file, len(false_positives))  # Mark the end of the false TICs

    # Clear astroquery cache
    Observations.clear_cache()

In [4]:


batch_size = 150 # Adjust the batch size as needed
batch_number = 5  # Change this to the batch number to continue from

for i in range(3):
    download_and_zip_lc_batches(batch_size, batch_number)
    batch_number = batch_number + 1

Processing and Saving Data Start...
Creating Observation Table
Attempting to download...
Error processing file ./mastDownload/HLSP/hlsp_qlp_tess_ffi_s0001-0000000031634579_tess_v01_llc/hlsp_qlp_tess_ffi_s0001-0000000031634579_tess_v01_llc.txt: No SIMPLE card found, this file does not appear to be a valid FITS file. If this is really a FITS file, try with ignore_missing_simple=True
Error processing file ./mastDownload/HLSP/hlsp_qlp_tess_ffi_s0001-0000000031943299_tess_v01_llc/hlsp_qlp_tess_ffi_s0001-0000000031943299_tess_v01_llc.txt: No SIMPLE card found, this file does not appear to be a valid FITS file. If this is really a FITS file, try with ignore_missing_simple=True
Error processing file ./mastDownload/HLSP/hlsp_qlp_tess_ffi_s0001-0000000038965512_tess_v01_llc/hlsp_qlp_tess_ffi_s0001-0000000038965512_tess_v01_llc.txt: No SIMPLE card found, this file does not appear to be a valid FITS file. If this is really a FITS file, try with ignore_missing_simple=True
Error processing file ./ma